In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torchmetrics.classification import MulticlassF1Score
from torch.utils.tensorboard import SummaryWriter

import tqdm
import os
import numpy as np

In [ ]:
# ----------------------------
# 환경 설정
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20
patience = 5
batch_size = 48
learning_rate = 1e-4

In [ ]:
# ----------------------------
# 데이터셋 및 분할
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)), # ConvNeXt base model expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root="data/Dataset_project4", transform=transform)

class_names = dataset.classes
num_classes = len(class_names)
writer = SummaryWriter()

# 80:20 train/test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)


In [ ]:
# ----------------------------
# 모델, 손실 함수, 옵티마이저 설정
# ----------------------------
# ConvNeXt 모델 불러오기 (v2_base 버전)
#model = models.convnext_v2_base(weights='IMAGENET1K_V1')
model = models.convnext_base(weights=models.ConvNeXt_Base_Weights.DEFAULT)

# 전이 학습을 위해 모델의 파라미터 고정
# for param in model.parameters():
#     param.requires_grad = False

# 분류기 레이어 교체
# ConvNeXt는 `classifier` 모듈 안에 `2`번째 레이어에 최종 출력이 있습니다.
num_ftrs = model.classifier[2].in_features
model.classifier[2] = nn.Linear(num_ftrs, num_classes)
model = model.to(device)
print(num_ftrs)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
f1_metric = MulticlassF1Score(num_classes=num_classes, average='macro').to(device)



In [ ]:
# ----------------------------
# 학습 루프
# ----------------------------
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    loop = tqdm.tqdm(train_loader, leave=True)
    for X_batch, y_batch in loop:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # 순전파
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    return running_loss / len(train_loader)



In [ ]:
# ----------------------------
# 평가 루프
# ----------------------------
def evaluate_model(model, test_loader, criterion, device, f1_metric):
    model.eval()
    running_loss = 0.0
    y_pred_prob = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        loop = tqdm.tqdm(test_loader, leave=True)
        for X_batch, y_batch in loop:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            running_loss += loss.item()
            
            probs, preds = torch.max(outputs, dim=1)
            y_pred_prob.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(y_batch.cpu().numpy())
            
    avg_loss = running_loss / len(test_loader)
    
    f1_metric.update(torch.tensor(y_pred).to(device), torch.tensor(y_true).to(device))
    f1_score = f1_metric.compute()

    y_pred_prob = np.array(y_pred_prob)
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)

    accuracy = np.mean(y_pred == y_true)

    return avg_loss, accuracy, f1_score.item()



In [ ]:
# ----------------------------
# 메인 학습 및 평가 루프
# ----------------------------
best_loss = float('inf')
epochs_no_improve = 0
early_stop = False

for epoch in range(num_epochs):
    if early_stop:
        print("Early stopping triggered!")
        break
        
    train_loss = train_model(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc, test_f1 = evaluate_model(model, test_loader, criterion, device, f1_metric)
    
    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}')

    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/test', test_loss, epoch)
    writer.add_scalar('Accuracy/test', test_acc, epoch)
    writer.add_scalar('F1 Score/test', test_f1, epoch)
    
    if test_loss < best_loss:
        best_loss = test_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_convnext_model.pth')
        print("Best model saved!")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            early_stop = True

writer.close()
print("Training complete.")